In [1]:
import cv2
import os
import numpy as np

In [2]:
for f in os.listdir():
    if f.endswith(".npy"):
        print(f)

In [3]:
X=[]
Y=[]

d={}
cnt=0

for f in os.listdir():
    if f.endswith(".npy"):
        imgs = np.load(f)
        print(imgs.shape,f)
        X.append(imgs)
        labels=np.ones((imgs.shape[0]))*cnt
        Y.append(labels)
        
        d[cnt]=f[:-4]
        cnt+=1

In [4]:
X=np.array(X)
Y=np.array(Y)

X=X.reshape((60,100,100,3))
Y=Y.reshape((60,))
print(X.shape,Y.shape)
print(d)

ValueError: cannot reshape array of size 0 into shape (60,100,100,3)

In [5]:
def dist(p1,p2):
    return np.sum((p1-p2)**2)**.5

def KNN(X,Y,test_point,k=5):
    
    d=[]
    m=X.shape[0]
    for i in range(m):
        current_dist=dist(test_point,X[i])
        d.append((current_dist,Y[i]))
    
    d=sorted(d)
    d=d[:k]
    d=np.array(d)
    
    d=d[:,1]
    
    uniq,cnts=np.unique(d,return_counts=True)
    idx=np.argmax(cnts)
    pred=uniq[idx]
    return int(pred)
    

In [ ]:
detector= cv2.CascadeClassifier('Datasets/FaceCascade/templatedata.xml')

camera =cv2.VideoCapture(0)

while True:
    b,img=camera.read()
    if b==False:
        continue
       
    faces = detector.detectMultiScale(img,1.3)
    if(len(faces)==0):
        continue
        
    
    for f in faces:
        f=faces[0]
        x,y,w,h =f 
        green=(0,255,0)
        cv2.rectangle(img,(x,y),(x+w,y+h),green,5)
        crop_face=img[y:y+h,x:x+w]
     
        small_face=cv2.resize(crop_face,(100,100))
        
        predicted_name=d[KNN(X,Y,small_face)]
        
        cv2.putText(img, predicted_name, (x,y-20), cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,255,255))
    cv2.imshow("image ",img)
    key=cv2.waitKey(1)&0xFF
    if key==ord('q'):
        break

        
camera.release()
cv2.destroyAllWindows()